In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from regpy.operators.convolution import ConvolutionOperator, GaussianBlur, ExponentialConvolution
from regpy.vecsps import UniformGridFcts
from regpy.solvers import TikhonovRegularizationSetting, RegularizationSetting, RegSolver
from regpy.solvers.linear.semismoothNewton import SemismoothNewton_bilateral, SemismoothNewton_nonneg, SemismoothNewtonAlphaGrid
from regpy.solvers.linear.tikhonov import TikhonovCG
from regpy.solvers.linear.primal_dual import PDHG
from regpy.solvers.linear.proximal_gradient import FISTA
import regpy.functionals as fct
from regpy.hilbert import L2
from regpy.stoprules import CountIterations
import logging

# Deconvolution with impulsive noise
We set up a 1D periodic deconvolution problem with impulsive noise. 
If we solve this problem by Tikhonov regularization with a quadratic data fidelity term, then the reconstruction is dominated by the effects of this impulsive noise. 

In [ ]:
n = 256
sigma = 0.002
alpha = 0.1
grid = UniformGridFcts((-1,1,n),periodic=True)
conv = GaussianBlur(grid,kernel_width=0.05)

x = grid.coords[0]
fdag = np.maximum(0,1-3*np.abs(x))
gobs = conv(fdag)

for j in range(5):
    k = np.random.randint(n)
    gobs[k] = np.random.randint(50)

setting = RegularizationSetting(conv,L2,L2)
solver = TikhonovCG(setting=setting, data = gobs, regpar = alpha,reltolx=1e-6)
fal,_ = solver.run()

fig, (ax1,ax2) =plt.subplots(1,2)
ax1.plot(x,gobs)
ax2.plot(x,fal,label='Tikh. sol')
ax2.plot(x,fdag,label='true')
ax2.legend()

# setting with Huber data fidelity term
We now set up a generalized Tikhonov regularization setting
$$
\hat{f} \in \mathrm{argmin}_f\left[\frac{1}{\alpha}H_{\sigma}(Tf-g^{\mathrm{obs}})+\|f\|^2\right]    
$$
where the Huber data fidelity term $H_{\sigma}$ is quadratic on coordinates in $[-\sigma,\sigma]$ and $|\cdot|-\sigma/2$ for large coordinates.

Moreover, we set up three solvers for this setting:
- FISTA
- Primal-Dual Hybrid Gradient (Chambolle-Pock) method
- Semismooth Newton method


In [ ]:
Sdat = (1./sigma)*fct.Huber(grid,sigma=sigma)
huber_setting = TikhonovRegularizationSetting(conv,L2,Sdat,alpha,data_fid_shift=gobs)
dual_setting = huber_setting.dualSetting()

solverFISTA = FISTA(dual_setting,logging_level=logging.DEBUG)
solverPDHG = PDHG(huber_setting,logging_level=logging.DEBUG)
solverSS = SemismoothNewton_bilateral(dual_setting,cgpar={'reltolx':1e-6,'tol':1e-6},logging_level=logging.DEBUG)
itPDHG = iter(solverPDHG)
itFISTA = iter(solverFISTA)

In [ ]:
from regpy.solvers.linear.admm import ADMM

otf = conv.fourier_multiplier
TikhInverse = ConvolutionOperator(grid, 1./(1.+np.abs(otf)**2))
solverADMM = ADMM(huber_setting,gamma=1,regularizedInverse=TikhInverse,logging_level=logging.DEBUG)
itADMM = iter(solverADMM)

In [ ]:
for j in range(20):
    f_ADMM,y_ADMM = next(itADMM)
fig, (ax1,ax2) = plt.subplots(1,2)
ax1.plot(f_ADMM,label = 'f_ADMM')
ax1.legend()
ax2.plot(y_ADMM,label='y_ADMM')
ax2.legend()

# FISTA
Run this cell until the duality gap is sufficiently small. 

In [ ]:
for j in range(20):
    p_FISTA,Tstar_pFISTA = next(itFISTA)
f_FISTA = huber_setting.dualToPrimal(Tstar_pFISTA,argumentIsOperatorImage=True)
"""
fig, (ax1,ax2) = plt.subplots(1,2)
ax1.plot(p_FISTA,label = 'p_FISTA')
ax1.legend()
ax2.plot(f_FISTA,label='f_FISTA')
ax2.legend()
"""

# PDHG
Run this cell until the duality gap is sufficiently small. 

In [ ]:
for j in range(20):
    f_PDHG,p_PDHG = next(itPDHG)
"""
fig, (ax1,ax2) = plt.subplots(1,2)
ax1.plot(p_PDHG,label = 'p_PDHG')
ax1.legend()
ax2.plot(f_PDHG,label='f_PDHG')
ax2.legend()
"""


# Semismooth Newton method

The method gives a solution that looks better than those of FISTA and PDHG, but the duality gap is much larger. 
(Possibly the effective regularization parameter is wrong!)

In [ ]:
#p_SS,Tstar_pSS = solverSS.run(stoprule=CountIterations(50))
#f_SS    = huber_setting.dualToPrimal(Tstar_pSS,argumentIsOperatorImage=True)
print('SSNewton duality gap - from dual:', huber_setting.dualityGap(dual=p_SS), 
      'from primal', huber_setting.dualityGap(primal=f_SS))

# Plot results of all methods for comparison

In [ ]:
fig,((ax11,ax12,ax13),(ax21,ax22,ax23)) = plt.subplots(2,3)

ax11.plot(p_FISTA,label='p FISTA'); ax11.legend()
ax12.plot(solverPDHG.pstar,label='p PDHG');   ax12.legend()
ax13.plot(p_SS, label = 'p_SS');     ax13.legend()

ax21.plot(f_FISTA,label='f FISTA'); ax21.legend()
ax22.plot(f_PDHG,label='f PDHG');   ax22.legend()
ax23.plot(f_SS, label = 'f_SS');     ax23.legend()